In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!unzip /content/drive/MyDrive/FYP/chest_xray_MobileNetV3_OriginalData>.zip -d /content/extracted_files

In [20]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras import layers, models, optimizers
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np


In [21]:
# Step 1: Set up paths for the dataset
dataset_dir = "/content/extracted_files/chest_xray_MobileNetV3_OriginalData"

# Define paths for training and testing
train_dir = os.path.join(dataset_dir, "/content/extracted_files/chest_xray_MobileNetV3_OriginalData/train")
test_dir = os.path.join(dataset_dir, "/content/extracted_files/chest_xray_MobileNetV3_OriginalData/test")



In [5]:
# Step 2: Load dataset without augmentation
img_height, img_width = 224, 224  # MobileNetV3 recommended input size


In [6]:
train_datagen = ImageDataGenerator(rescale=1.0 / 255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode="binary",
    subset="training",
)

Found 4187 images belonging to 2 classes.


In [7]:
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode="binary",
    subset="validation",
)

Found 1045 images belonging to 2 classes.


In [8]:
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode="binary",
    shuffle=False,
)

Found 624 images belonging to 2 classes.


In [9]:
# Step 3: Compute class weights
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights_dict = dict(enumerate(class_weights))
print("Class Weights:", class_weights_dict)


Class Weights: {0: 1.938425925925926, 1: 0.6738010943031864}


In [10]:
# Step 4: Define MobileNetV3 model
base_model = MobileNetV3Small(
    input_shape=(img_height, img_width, 3),
    include_top=False,
    weights="imagenet",
)


4334752/4334752 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [11]:
# Freeze base model
base_model.trainable = False


In [12]:
# Add custom layers on top of MobileNetV3
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')  # Binary classification
])

In [13]:
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ MobileNetV3Small (Functional)        │ (None, 7, 7, 576)           │         939,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 576)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,013,105 (3.86 MB)

 Trainable params: 73,985 (289.00 KB)

 Non-trainable params: 939,120 (3.58 MB)

In [14]:
# Compile the model
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.001),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)

In [15]:
# Step 5: Train the model with class weights
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10,  # Adjust as needed
    class_weight=class_weights_dict,
    verbose=1,
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


131/131 ━━━━━━━━━━━━━━━━━━━━ 128s 869ms/step - accuracy: 0.5053 - loss: 0.7207 - val_accuracy: 0.2574 - val_loss: 0.7268
Epoch 2/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 155s 987ms/step - accuracy: 0.5041 - loss: 0.6946 - val_accuracy: 0.2574 - val_loss: 0.7053
Epoch 3/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 114s 855ms/step - accuracy: 0.5381 - loss: 0.6865 - val_accuracy: 0.7522 - val_loss: 0.6861
Epoch 4/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 113s 844ms/step - accuracy: 0.4886 - loss: 0.6992 - val_accuracy: 0.6057 - val_loss: 0.6926
Epoch 5/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 147s 885ms/step - accuracy: 0.4906 - loss: 0.6947 - val_accuracy: 0.7407 - val_loss: 0.6844
Epoch 6/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 114s 841ms/step - accuracy: 0.5085 - loss: 0.7014 - val_accuracy: 0.7407 - val_loss: 0.6747
Epoch 7/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 111s 814ms/step - accuracy: 0.5528 - loss: 0.6953 - val_accuracy: 0.3522 - val_loss: 0.6969
Epoch 8/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 113s 844ms/step - accuracy: 0.5806 - loss: 0.68

In [16]:
# Step 6: Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator, verbose=1)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


20/20 ━━━━━━━━━━━━━━━━━━━━ 14s 673ms/step - accuracy: 0.7448 - loss: 0.6749
Test Accuracy: 70.99%


In [17]:
# Predictions and classification report
test_generator.reset()
y_pred = (model.predict(test_generator) > 0.5).astype("int32")
y_true = test_generator.classes


20/20 ━━━━━━━━━━━━━━━━━━━━ 20s 855ms/step


In [18]:
print("\nClassification Report:")
print(classification_report(
    y_true,
    y_pred,
    target_names=test_generator.class_indices.keys(),
    zero_division=0,  # Suppress undefined metric warning
))



Classification Report:
              precision    recall  f1-score   support

      NORMAL       0.58      0.79      0.67       234
   PNEUMONIA       0.84      0.66      0.74       390

    accuracy                           0.71       624
   macro avg       0.71      0.73      0.71       624
weighted avg       0.75      0.71      0.71       624



In [19]:
print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred))


Confusion Matrix:
[[186  48]
 [133 257]]
